In [5]:
#import config
import math
import copy
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F

# DEVICE = config.device
device =   "cuda" if torch.cuda.is_available() else "cpu"; # 检查是否有可用的GPU,否则使用CPU
print("Using {} device".format(device)); # 打印使用的设备类型
"""
Transformer模型中前馈神经网络的实现。
"""
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        位置前馈神经网络初始化函数
        参数:
            d_model: 模型的输入维度
            d_ff: 前馈神经网络中间层的维度
            dropout: dropout概率，默认为0.1
        """
        super(PositionwiseFeedForward, self).__init__()
        # Linear层，将输入维度从d_model映射到d_ff  
        self.w_1 = nn.Linear(d_model, d_ff)  # 第一个线性层，将维度从d_model扩展到d_ff
        self.w_2 = nn.Linear(d_ff, d_model)  # 第二个线性层，将维度从d_ff压缩回d_model
        self.dropout = nn.Dropout(dropout)    # dropout层，用于防止过拟合

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))  # 先通过第一个线性层，然后应用ReLU激活函数，再经过dropout，最后通过第二个线性层返回结果
    


Using cuda device


In [ ]:
"""
为每个输入位置添加一个唯一的位置编码
这个编码会被添加到词向量中，使模型能够理解位置信息
参考论文《Attention is All You Need》中的公式:
# 偶数维度使用sin函数，奇数维度使用cos函数
PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))
PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
其中 pos 是位置，i 是维度索引，d_model 是词向量的维度

"""
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化一个size为 max_len(设定的最大长度)×embedding维度 的全零矩阵
        # 来存放所有小于这个长度位置对应的positional embedding
        pe = torch.zeros(max_len, d_model, device=device)
        print("pe shape:", pe.shape)  # pe shape: torch.Size([5000, 512])
        print("pe:", pe);
        # 生成一个位置下标的tensor矩阵(每一行都是一个位置下标)
        position = torch.arange(0., max_len, device=device).unsqueeze(1)
        print("PositionalEncoding: d_model={}, max_len={}".format(d_model, max_len));
        print("position shape:", position.shape)  # position shape: torch.Size([5000, 1])
        # 这里幂运算太多，我们使用exp和log来转换实现公式中pos下面要除以的分母（由于是分母，要注意带负号）
        div_term = torch.exp(torch.arange(0., d_model, 2, device=device) * -(math.log(10000.0) / d_model))

        # 根据公式，计算各个位置在各embedding维度上的位置纹理值，存放到pe矩阵中
        # 偶数维度使用sin函数，奇数维度使用cos函数  
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 加1个维度，使得pe维度变为：1×max_len×embedding维度
        # (方便后续与一个batch的句子所有词的embedding批量相加)
        pe = pe.unsqueeze(0)
        # 将pe矩阵以持久的buffer状态存下(不会作为要训练的参数)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 将一个batch的句子所有词的embedding与已构建好的positional embeding相加
        # (这里按照该批次数据的最大句子长度来取对应需要的那些positional embedding值)
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

 

In [15]:
# 实例化FeedForward对象
d_model=512;

d_ff=2048; 

h=8;

dropout=0.1;

#ff = PositionwiseFeedForward(d_model, d_ff, dropout).to(device);


# 实例化PositionalEncoding对象
position = PositionalEncoding(d_model, dropout).to(device);


# 实例化Transformer模型对象

print(""" --- IGNORE ---""");
print(position.pe.shape);

pe shape: torch.Size([5000, 512])
pe: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
PositionalEncoding: d_model=512, max_len=5000
position shape: torch.Size([5000, 1])
pe after even: tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  0.0000e+00,  8.2186e-01,  ...,  0.0000e+00,
          1.0366e-04,  0.0000e+00],
        [ 9.0930e-01,  0.0000e+00,  9.3641e-01,  ...,  0.0000e+00,
          2.0733e-04,  0.0000e+00],
        ...,
        [ 9.5625e-01,  0.0000e+00,  9.3594e-01,  ...,  0.0000e+00,
          4.9515e-01,  0.0000e+00],
        [ 2.7050e-01,  0.0000e+00,  8.2251e-01,  ...,  0.0000e+00,
          4.9524e-01,  0.0000e+00],
        [-6.6395e-01,  0.0000e+00,  9.7326e-04,  ...,  0.0000e+0